## Setup

In [1]:
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import random
# import cudf as pd
import tensorflow as tf
import isuelogit as isl
import glob

from src.aesuelogit.models import UtilityFunction, AESUELOGIT, NGD
from src.aesuelogit.visualizations import plot_predictive_performance
from src.aesuelogit.networks import load_k_shortest_paths, read_paths, build_fresno_network, \
    Equilibrator, sparsify_OD, ColumnGenerator, read_OD

from src.aesuelogit.etl import get_design_tensor, get_y_tensor, data_curation

In [2]:
# Path management
main_dir = str(Path(os.path.abspath('')).parents[1])
os.chdir(main_dir)
print('main dir:', main_dir)

isl.config.dirs['read_network_data'] = "input/network-data/fresno/"

main dir: /Users/pablo/OneDrive/data-science/github/aesuelogit


In [3]:
# Seed for reproducibility
_SEED = 2022

np.random.seed(_SEED)
random.seed(_SEED)
tf.random.set_seed(_SEED)

## Build Fresno network

In [4]:
fresno_network = build_fresno_network()

## Read OD matrix

In [5]:
# TODO: specify path to read OD matrix
read_OD(network=fresno_network, sparse=True)

Matrix Q (1789, 1789) read in 0.1[s] with sparse format
66266.3 trips were loaded among 6970 o-d pairs


## Read paths

In [6]:
read_paths(network=fresno_network, update_incidence_matrices=True, filename = 'paths-full-model-fresno.csv')

Matrix D (2413, 18289) generated in 21.2[s]               

Matrix M (6970, 18289) generated in 7.9[s]               

Matrix C (18289, 18289) generated in 5.0[s]               



## Read spatiotemporal data

In [7]:
folderpath = isl.config.dirs['read_network_data'] + 'links/spatiotemporal-data/'
df = pd.concat([pd.read_csv(file) for file in glob.glob(folderpath + "*fresno-link-data*")], axis=0)

df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df = df[df['date'].dt.dayofweek.between(0, 4)]
# df = df[df['date'].dt.year == 2019]

df['period'] = df['date'].astype(str) + '-' + df['hour'].astype(str)
df['period'] = df.period.map(hash)

## Utility function

In [8]:
features_Z = ['speed_sd', 'median_inc', 'incidents', 'bus_stops', 'intersections']
# features_Z = ['speed_sd']
# features_Z = []

utility_function = UtilityFunction(features_Y=['tt'],
                                   features_Z=features_Z,
                                   signs={'tt': '-', 'speed_sd': '-', 'median_inc': '+', 'incidents': '-',
                                          'bus_stops': '-', 'intersections': '-'},
                                   # initial_values={'tt': 0, 'median_inc': 0, 'incidents': 0}
                                   )

utility_function.constant_initializer(0)

## Data processing

In [9]:
df['tt_avg'] = np.divide(df['length'],df['speed_hist_avg'],
                         out=np.zeros_like(df['length']), where=df['speed_hist_avg']!=0)

df['tt_ff'] = np.divide(df['length'],df['speed_ref_avg'],
                         out=np.zeros_like(df['length']), where=df['speed_ref_avg']!=0)

In [10]:
df = data_curation(df)

In [11]:
n_links = len(fresno_network.links)
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df['year'] = df.date.dt.year
X, Y = {}, {}

In [12]:
df.query('year == 2019')[['counts', 'tt_ff', 'tt_avg', 'tf_inrix']].describe()

,counts,tt_ff,tt_avg,tf_inrix
count,9704.000000,166497.000000,166497.000000,166497.000000
mean,1880.469281,0.003396,0.005273,0.192740
std,791.369747,0.003330,0.005555,0.224399
min,3.000000,0.000000,0.000000,0.000000
25%,1361.000000,0.000000,0.000000,0.000000
50%,1787.000000,0.003430,0.005364,0.144000
75%,2302.700000,0.004736,0.007081,0.272000
max,4807.000000,0.038373,0.068856,4.605000


In [13]:
df.query('year == 2020')[['counts', 'tt_ff', 'tt_avg', 'tf_inrix']].describe()

,counts,tt_ff,tt_avg,tf_inrix
count,9290.000000,159258.000000,159258.000000,159258.000000
mean,1822.742196,0.003343,0.005127,0.188521
std,795.855464,0.003259,0.005394,0.219121
min,31.000000,0.000000,0.000000,0.000000
25%,1275.125000,0.000000,0.000000,0.000000
50%,1715.000000,0.003384,0.005140,0.142000
75%,2242.750000,0.004736,0.006789,0.263000
max,4798.000000,0.033796,0.067593,3.775000


In [14]:
# Normalization of features to range [0,1]
df[features_Z + ['tt_avg'] + ['tt_ff']] \
    = preprocessing.MinMaxScaler().fit_transform(df[features_Z + ['tt_avg'] + ['tt_ff']])

In [15]:
for year in sorted(df['year'].unique()):
    df_year = df[df['year'] == year]

    n_days, n_hours = len(df_year.date.unique()), len(df_year.hour.unique())

    # TODO: Add an assert to check the dataframe is properly sorted before reshaping it into a tensor

    traveltime_data = get_y_tensor(y=df_year[['tt_avg']], n_links=n_links, n_days=n_days, n_hours=n_hours)
    flow_data = get_y_tensor(y=df_year[['counts']], n_links=n_links, n_days=n_days, n_hours=n_hours)

    Y[year] = tf.concat([traveltime_data, flow_data], axis=3)

    X[year] = get_design_tensor(Z=df_year[['tt_avg'] + features_Z], n_links=n_links, n_days=n_days, n_hours=n_hours)

    # For further internal processing, free flow travel times are stacked as the first feature of matrix X
    tt_ff = get_design_tensor(Z=df_year[['tt_ff']], n_links=n_links, n_days=n_days, n_hours=n_hours)
    X[year] = tf.concat([tt_ff, X[year]], axis=3)

2022-05-24 11:03:28.004836: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Training

In [16]:
# Prepare the training and validation dataset

X_train, X_val, Y_train, Y_val = X[2019], X[2020], Y[2019], Y[2020]

X_train, X_val, Y_train, Y_val = [tf.constant(i) for i in [X_train, X_val, Y_train, Y_val]]

In [17]:
_EPOCHS = 10
_BATCH_SIZE = 4
_LR = 5e-1  # Default is 1e-3. With 1e-1, training becomes unstable

In [18]:
# models = dict(zip(['m1', 'm2', 'm3', 'm4'], True))
models = dict.fromkeys(['m1', 'm2', 'm3', 'm4'], False)
models['m1'] = True
# models['m2'] = True
# models['m3'] = True
# models['m4'] = True

In [19]:
# optimizer = tf.keras.optimizers.Adagrad(learning_rate=_LR)
optimizer = tf.keras.optimizers.Adam(learning_rate=_LR)

In [20]:
equilibrator = Equilibrator(
    network=fresno_network,
    # paths_generator=paths_generator,
    utility_function=utility_function,
    max_iters=100,
    method='fw',
    iters_fw=50,
    accuracy=1e-4,
)

column_generator = ColumnGenerator(equilibrator=equilibrator,
                                   utility_function=utility_function,
                                   n_paths=0,
                                   ods_coverage=0.1,
                                   ods_sampling='sequential',
                                   # ods_sampling='demand',
                                   )

In [21]:
train_losses_dfs = {}
val_losses_dfs = {}

In [ ]:
if models['m1']:
    print('\nmodel 1: Benchmark of aesuelogit and isuelogit')

    # optimizer = NGD(learning_rate=_LR)

    # Model 1 (Utility only)
    model_1 = AESUELOGIT(
        key='model_1',
        network=fresno_network,
        dtype=tf.float64,
        trainables={'theta': True, 'theta_links': False, 'psc_factor': False, 'q': False, 'alpha': False, 'beta': False},
        equilibrator = equilibrator,
        column_generator = column_generator,
        utility_function=utility_function,
        inits={
            'q': fresno_network.q.flatten(),
            # 'q': 10*np.ones_like(fresno_network.q.flatten()),
            'theta': np.array(list(utility_function.initial_values.values())),
            'beta': np.array([4]),
            'alpha': np.array([0.15])
            # 'alpha': np.array([1 for link in fresno_network.links])
        },
    )

    train_losses_dfs['model_1'], val_losses_dfs['model_1'] = model_1.train(
        X_train, Y_train, X_val, Y_val,
        optimizer=optimizer,
        batch_size=_BATCH_SIZE,
        lambdas={'od': 0, 'theta': 0, 'tt': 0, 'flow': 1, 'bpr': 0},
        epochs=_EPOCHS)

    plot_predictive_performance(train_losses=train_losses_dfs['model_1'], val_losses=val_losses_dfs['model_1'])

    print(f"theta = {dict(zip(utility_function.true_values.keys(), list(model_1.theta.numpy())))}")
    print(f"alpha = {model_1.alpha}, beta  = {model_1.beta}")
    print("Avg abs diff between observed and estimated OD:",
          f"{np.mean(np.abs(model_1.q - fresno_network.q.flatten()))}")


model 1: Benchmark of aesuelogit and isuelogit

Epoch: 0
0: train_loss=2.3e+06,  val_loss=2.4e+06, train_loss tt=0.015, val_loss tt=0.014, train_loss flow=2.3e+06, val_loss flow=2.4e+06, train_loss bpr=0.0061, val_loss bpr=0.005, avg abs diff demand =8e-16, theta = [0. 0. 0. 0. 0. 0.], psc_factor = 0.0, avg alpha = 0.15, avg beta = 4, time:  14

Epoch: 1
1: train_loss=1.7e+06,  val_loss=1.7e+06, train_loss tt=0.0062, val_loss tt=0.0057, train_loss flow=1.7e+06, val_loss flow=1.7e+06, train_loss bpr=0.0061, val_loss bpr=0.005, avg abs diff demand =8e-16, theta = [-1.5658 -0.8841  0.     -2.4641  0.     -1.7911], psc_factor = 0.0, avg alpha = 0.15, avg beta = 4, time:  17


In [ ]:
if models['m2']:
    print('\nmodel 2: OD + utility estimation with historic OD')

    model_2 = AESUELOGIT(
        key='model_2',
        network=fresno_network,
        dtype=tf.float64,
        trainables={'theta': True, 'theta_links': False, 'q': True, 'alpha': False, 'beta': False},
        equilibrator=equilibrator,
        column_generator=column_generator,
        utility_function=utility_function,
        inits={
            'q': fresno_network.q.flatten()[np.newaxis,:],
            'theta': np.array(list(utility_function.initial_values.values())),
            'beta': np.array([4]),
            'alpha': np.array([0.15])
        },
    )

    train_losses_dfs['model_2'], val_losses_dfs['model_2'] = model_2.train(
        X_train, Y_train, X_val, Y_val,
        optimizer=optimizer,
        batch_size=_BATCH_SIZE,
        lambdas={'od': 1, 'theta': 0, 'tt': 1, 'flow': 1, 'bpr': 0},
        epochs=_EPOCHS)

    plot_predictive_performance(train_losses=train_losses_dfs['model_2'], val_losses=val_losses_dfs['model_2'])

    print(f"theta = {dict(zip(utility_function.true_values.keys(), list(model_2.theta.numpy())))}")
    print(f"alpha = {model_2.alpha}, beta  = {model_2.beta}")
    print("Avg abs diff between observed and estimated OD:",
          f"{np.mean(np.abs(model_2.q - fresno_network.q.flatten()))}")

In [ ]:
if models['m3']:

    print('\nmodel 3: ODLUE + link performance parameters without historic OD matrix')

    # Model 3 (Utility, ODE, link performance parameters, no historic OD)
    model_3 = AESUELOGIT(
        key='model_3',
        network=fresno_network,
        dtype=tf.float64,
        trainables={'theta': True, 'theta_links': False, 'psc_factor': True, 'q': True, 'alpha': True, 'beta': True},
        equilibrator=equilibrator,
        column_generator=column_generator,
        utility_function=utility_function,
        inits={
            'q': fresno_network.q.flatten()[np.newaxis,:],
            'theta': np.array(list(utility_function.initial_values.values())),
            'beta': np.array([4]),
            'alpha': np.array([0.15])
        },
    )

    train_losses_dfs['model_3'], val_losses_dfs['model_3'] = model_3.train(
        X_train, Y_train, X_val, Y_val,
        optimizer=optimizer,
        batch_size=_BATCH_SIZE,
        lambdas={'od': 0, 'theta': 0, 'tt': 1, 'flow': 1, 'bpr': 0},
        epochs=_EPOCHS)

    plot_predictive_performance(train_losses=train_losses_dfs['model_3'], val_losses=val_losses_dfs['model_3'])

    print(f"theta = {dict(zip(utility_function.true_values.keys(), list(model_3.theta.numpy())))}")
    print(f"alpha = {model_3.alpha}, beta  = {model_3.beta}")
    print("Avg abs diff between observed and estimated OD:",
          f"{np.mean(np.abs(model_3.q - fresno_network.q.flatten()))}")

In [ ]:
if models['m4']:
    print('\nmodel 4: Time specific utility and OD, link performance parameters, no historic OD')

    # Model 4 (Time specific Utility and ODE, link performance parameters, no historic OD)
    model_4 = AESUELOGIT(
        key='model_4',
        network=fresno_network,
        dtype=tf.float64,
        trainables={'theta': True, 'theta_links': False, 'psc_factor': True, 'q': True, 'alpha': True, 'beta': True},
        equilibrator=equilibrator,
        column_generator=column_generator,
        utility_function=utility_function,
        inits={
            'q': np.repeat(fresno_network.q.flatten()[np.newaxis, :], n_hours, axis=0),
            'theta': np.repeat(np.array(list(utility_function.initial_values.values()))[np.newaxis, :], n_hours,
                               axis=0),
            'beta': np.array([4]),
            'alpha': np.array([0.15])
        },
    )
    train_losses_dfs['model_4'], val_losses_dfs['model_4'] = model_4.train(
        X_train, Y_train, X_val, Y_val,
        optimizer=optimizer,
        batch_size=_BATCH_SIZE,
        lambdas={'od': 0, 'theta': 0, 'tt': 1, 'flow': 1, 'bpr': 0},
        epochs=_EPOCHS)

    plot_predictive_performance(train_losses=train_losses_dfs['model_4'], val_losses=val_losses_dfs['model_4'])

    print(f"features = {utility_function.features}")
    print(f"theta = {model_4.theta.numpy()}")
    print(f"alpha = {model_4.alpha}, beta  = {model_4.beta}")
    print("Avg abs diff between observed and estimated OD:",
          f"{np.mean(np.abs(model_4.q - fresno_network.q.flatten()))}")